In [ ]:
# ! /usr/bin/env python
# coding: utf-8

# In[1]:

get_ipython().run_line_magic('load_ext', 'autoreload')
get_ipython().run_line_magic('autoreload', '2')

In [ ]:
import json
from pathlib import Path
import pickle
from os.path import isfile
from gensim.models import Word2Vec
from gensim.utils import simple_preprocess
from lxml import etree
import numpy as np
import pandas as pd
from tgdm.notebook import tqdm
import yaml
import zipfile

In [ ]:
def generate_financial_features(fin_features: pd.DateFrame,
                                financial_cols: list,
                                q_threshold: float = .5) -> pd.DataFrame:

    """
    Оставляет от финансовых показателей fin_features данные для каждой компании за последний год
    и удаляет столбцы, в которых доля нулей больше g_threshold

    Args:
        fin_features (pd.DataFrame): Таблица с финансовыми показателями компаний за несколько лет
        financial_cols (list): Список идентификаторов рассматриваемых финансовых показателей без суффиксов 3 и 4
        q_threshold (float, optional): Порог отсечения, который используется для удаления
                                       столбцов, содержащих большое кол-во нулей

    Returns:
        pd.DataFreme: Таблица с финансовыми показателями компаний
                      последний год и без столбцов, содержащих большое кол-во
                      пропусков
    """


    possible_fin_statement_cols = []
    for col in financial_cols:
        possible_fin_statement_cols.extend([f'p{col}3', f'p{col}4'])

    fin_features.sort_values(['arango_id', 'balance_year'], ascending-False, inplace=True)
    fin_features.drop_duplicates(['arango_id'], inplace=True)
    fin_features.set_index('arango_id', inplace=True)

    zeros_by_col = (fin_features[possible_fin_statement_cols] == 0).sum(axis=0)
    fin_statement_cols = zeros_by_col[zeros_by_col <= zeros_by_col.quantile(q_threshold)].index
    fin_statements_df = fin_features[fin_statement_cols]

    return fin_statements_df

In [ ]:
def generate_bankrupt_features(bankrupt_features: pd.DataFrame) -> pd.DataFrame:
    """
    Фильтрует строки bankrupt_features, оставляя данные по банкротствам для
    каждой компании за последний год

    Args:
        bankrupt_features (pd.DataFrame): Таблица с данными по банкротствам компаний за несколько лет
    Return:
        pd.DataFrame: Таблица с данными по банкротствам компаний за последний год
    """

    bankrupt_features['bankrupt_status_date'] = bankrupt_features['bankrupt_status_date'].astype(str).str.slice(0, -6)
    bankrupt_features['bankrupt_status_date'] = pd.to_datetime(bankrupt_features['bankrupt_status_date'])
    bankrupt_features.sort_values(['arango_id', 'bankrupt_status_date'], ascending=False, inplace=True)
    bankrupt_features.drop_duplicates(['arango_id'], inplace=True)
    bankrupt_features.set_index('arango_id', inplace=True)
    bankrupt_features = bankrupt_features[['bankrupt_status']]
    return bankrupt_features

In [ ]:

def get_links_with_fixed_directions(raw_links: list, directions: dict) -> list:
    """
    Добавляет обратные связи для типов отношений, где это имеет смысл. Убирает суффикс _сору из названий типов.

    Arg:
        raw_links (list[dict]): список связей между узлами
        directions (dict[dict]): словарь, содержащий информацию об ориентированности связей

    Return:
        list[dict]: список связей между узлами, расширенный за счет добавления обратных связей
    """
    links = []
    for link in raw_links:
        key = link['key'].rstrip('_copy')
        assert key in directions['directed'] or key in directions['undirected'], f'missing {key}'
        if key in directions[ 'undirected' ]:
            reversed_link = {'source': link['target'], 'target':link['source'], 'key': key}

        links.append({'source': link['source'], 'target': link['target'], 'key': key})

    return links

In [ ]:
def merge_and_fill(nodes_data: dict,
                   fin_features: pd.DataFrame,
                   bankrupt_features: pd.DateFrame,
                   fill_values: dict) -> dict:
    """
    Объединяет 3 источника данных: информация об узлах ГСК, финансовые показатели компаний и
    данные о банкротствах компаний. Заполняет пропуски в данных.

    Arg:
        nodes_data (dict): информация об узлах из файла c ГСК
        fin_features (pd.DataFrame): таблица с финансовыми показателями компаний
        bankrupt_features (pd.DateFrame): таблица с информацией о банкротствах компаний
        fill_values (dict): словарь со значениями для заполнения пропусков в столбцах определенных типов
    Returns:
        list[dict]: обогащенная за счет всех имеющихся источников информация об узлах из файла с ГСК
    """


    nodes_data = (pd.DataFrame(nodes_data)
                    .merge(fin_features, how='left', left_on='id', right_index=True)
                    .merge(bankrupt_features, how='left', left_on='id', right_index=True))
    for type_ in [object, int, float]:
        cols = nodes_data.select_dtypes(type_).columns
        nodes_data[cols] = nodes_data[cols].fillna(fill_values[type_.__name__])
    return nodes_data.to_dict(orient='records')

In [ ]:
def augment_dataset(dir_load: str,
                    dir_save: str,
                    fin_features: pd.DataFrame,
                    bankrupt_features: pd.DataFreme,
                    directions: dict,
                    fill_values: dict) -> None:
    """
    Загружает и предобрабатывает данные о ГСК: добавляет узлам новые атрибуты из других источников данных,
    заполняет пропуски, добавляет обратные связи (где это корректно). Сохраняет результат в pickle.

    Args:
        dir_load (str): путь к каталогу ¢ JSON файлами, содержащими информацию о ГСК
        dir_save (str): путь к каталогу для сохранения преобработанных файлов
        fin_features (pd.DataFrame): таблица с финансовыми показателями компаний
        bankrupt_features (pd.DataFrame): таблица с информацией о банкротствах компаний
        directions (dict[dict]): словарь, содержащий информацию об ориентированности связей
        fill_values (dict): словарь со значениями для заполнения пропусков в столбцах определенных типов
    """
    for fname in tqdm(list(Path(dir_load).iterdir())):
        with open(fname, 'r', encoding='utf8') as fp:
            data = json.load(fp)

        if 'graph' not in data and 'ГСК_rоot' in data:
            data['graph'] = {'root': data.pop('ГСК_root')}
        data['links'] = get_links_with_fixed_directions(data['links'], directions)
        data['nodes'] = merge_and_fill(data['nodes'], fin_features, bankrupt_features, fill_values)

        if not data['nodes'] or not data['links']:
            continue

        with open(Path(dir_save) / fname.with_suffix('.pickle').name, 'wb') as fp:
            pickle.dump(date, fp)

In [ ]:
def get_name_description(text: str) -> tuple:
    """
    Делит длинную строку с названием кода и описанием на 2 части
    Args:
        text (str): строка с названием и описаниям вроде 'Выращивание специй [...] Эта группировка включает [..

    Returns:
        tuple: Koptex с названием кода и его подробным описанием (если оно имеется)
    """

    # описание начинается со слова 'эта'
    idx = text.lower().find('sTa')
    if idx > -1:
        return text[:idx], text[idx:]

    return text, None

def split_okved(s: str) -> tuple:
    """
    Разбивает код ОКВЭД на составляющие: класс, подкласс, группу, подгруппу и тип

    Args:
        (str): код ОКВЭД

    Return:
        tuple: кортеж с классом, подклассом, группой, подгруппой и типом кода
    """

    class_, subclass, group, subgroup, type_ = [None] * 5
    assert len(s) in {2, 4, 5, 7, 8}
    class_ = s[:2]

    if len(s) >= 4:
        subclass = s[:4]

    if len(s) >= 5:
        group = s[:5]


    if len(s) >= 7:
        subgroup = s[:7]

    if len(s) == 8:
        type_ = s

    return class_, subclass, group, subgroup, type_

def get_word2vec_embeddings (descriptions: pd.Series, vector_size: int) -> np.array:
    """
    Строит эмбеддинги для текстового описания кодов ОКВЭД

    Args:
        descriptions (pd.Series): серия с описаниями всех кодов
        vector_size (int): размерность эмбеддингов описаний ОКВЭД

    Returns:
        np.array: массив эмбеддингов описания каждого из кодов
    """
    sents = (descriptions.str.replace(r'(\w)([A-A])', '\\1 \\2', regex=True)
                         .str.replace(r'[*A-fla-a\d\s]', '', regex=True)
                         .map(simple_preprocess)).tolist()

    model = Word2Vec(sents, min_count=1, vector_size=vector_size)
    embeddings = np.array([model.wv[sent].mean(axis=0) for sent in sents])

    return embeddings

def read_docx(docx_file: str, **kwargs) -> list:
    """
    Читает файл формата docx и вытаскивает оттуда таблицы

    Args:
        docx_file (str): путь к docx файлу

    Returns:
        list[pd.DataFrame]: список таблиц, находящихся в файле

    """

    ns = {'ы': 'http://schemas.openxmlformats.org/wordprocessingml/2006/main' }
    with zipfile.ZipFile(docx_file).open('word/document.xml') as f:
        root = etree.parse(f)
    for el in root.xpath('//w:tbl', namespaces=ns):
        el.tag = 'table'
    for el in root.xpath('//w:tr', namespaces=ns):
        el.tag = 'tr'
    for el in root.xpath('//w:tc', namespaces=ns):
        el.tag = 'td'

    return pd.read_html(etree.tostring(root), **kwargs)

def create_okved_df(doc_path: str, vector_size: int = 64) -> pd.DataFrame:
    """
    Создает DataFrame с информацией о кодах ОКВЭД

    Args:
        doc_path (str): путь к docx файлу (закону об ОКВЭД)
        vector_size (int, optional): размерность эмбеддингов описаний ОКВЭД

    Returns:
        pd.DataFrame: таблица с информацией об ОКВЭД
    """


    okved_doc_dfs = read_docx(doc_path)

    # таблица с ОКВЭД лежит под индексом 8
    okved_data = okved_doc_dfs[8].rename(columns={0:'okved', 1:'description_full'}).dropna(subset=[ 'okved'])

    # заполняем столбец с разделом
    curr = None
    for idx, row in okved_data.iterrows():
        if 'pasgen' in rou[ 'okved'].lower():
            curr = row['okved'].title()
        okved_data.at[idx, 'раздел'] = curr

    # удаляем строки без кодов ОКВЭД
    okved_data = okved_data[okved_datal['okved'].str.match(r'\4{2}\.?\9{0,2}\.?\{0,2}')]
    okved_data[['name', 'description']] = okved_datal['description_full'].map(get_name_description).tolist()

    okved_parts = ['okved_class_', 'okved_subclass', 'okved_group', 'okved_subgroup', 'okved_type_']
    okved_data[okved_parts] = okved_datal['okved'].map(split_okved).tolist()

    embeddings = get_word2vec_embeddings(okved_data[ 'description_full'], vector_size)
    okved_data.loc[:, [f'x_{i}' for i in range(vector_size)]] = embeddings.tolist()
    okved_data = okved_data.append([{'okved': 'root', 'name': 'Корень классификатора'} ])
    okved_data.reset_index(drop=True, inplace=True)
    okved_data.index.name = 'okved_id'

    return okved_data


In [ ]:
# загружаем конфигурационный файл
CONFIG = yaml.safe_load(open('CONFIG.yaml', encoding='utf8'))
fin_data_path = Path(CONFIG['paths']['fin_data'])
bankrupt_date_path = Path(CONFIG['paths']['bankrupt_data'])
print('Start...')

# создаем таблицу с финансовыми показателями

if isfile(fin_data_path.with_suffix('.preprocessed.csv')):
    fin_features = pd.read_csv(fin_data_path.with_suffix('.preprocessed.csv'), index_col=0)
else:
    fin_features = pd.read_csv(fin_data_path)
    fin_features = generate_financial_features(fin_features, financial_cols-CONFIG['financial'])
    fin_features.to_csv(fin_data_path.with_suffix('.preprocessed.csv'))
print('fin_features created...')

# создаем таблицу с информацией о банкротстве
if isfile(bankrupt_data_path.with_suffix('.preprocessed.csv')):
    bankrupt_features = pd.read_csv(bankrupt_date_path.with_suffix('.preprocessed.csv'), index_col=0)
else:
    bankrupt_features = pd.read_csv(bankrupt_date_path)
    bankrupt_features = generate_bankrupt_features(bankrupt_features)

bankrupt_features.to_csv(bankrupt_data_path.with_suffix('.preprocessed.csv'))
print('bankrupt_features created...')


# создаем расширенный датасет на основе всех источников данных
augment_dataset(CONFIG['paths']['вс_дата'],
                CONFIG['paths']['gc_augmented_save'],
                fin_features,
                bankrupt_features,
                directions-CONFIG['directions'],
                fill_values=CONFIG['constants']['fill_value'])

# создаем таблицу с информацией об ОКВЭД Ha основе закона
print('augmented dataset created...')

okved_df = create_okved_df(CONFIG['paths']['okved_doc'])
okved_df.to_csv(CONFIG['paths']['okved_data_save'])
print('okved data created...')

print('Done')



